In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib import gridspec
from datetime import datetime
import ipywidgets as widgets
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import copy
from datetime import datetime
import random

#Lectura de identificadores
#Como los archivos descargados siguen un formato que involucra el identificador, es util tener a mano estos datos para agrupar los datos

f = open("Datos/ID_Estaciones.txt", "r")
ids = []
lines = f.readlines()
ids_dict = {}

for line in lines:
    ids.append(line.split(",")[2].split("\n")[0])
    ids_dict[line.split(",")[2].split("\n")[0]] = {}

print(ids)
print(ids_dict)
labels = copy.deepcopy(ids)

bigDf = pd.DataFrame()
for id in ids:
    df = pd.read_excel(id+"_agrupado.xlsx", sheet_name="Sheet1")
    if(bigDf.size == 0):
        bigDf = pd.concat([df.iloc[:,1:]], ignore_index = True)
    else:
        bigDf = pd.concat([bigDf,df.iloc[:,1:]], ignore_index = True)

bigDf["momento"] = pd.to_datetime(bigDf["momento"], format="%d-%m-%Y %H:%M:%S")
bigDf.dropna()

['510005', '400009', '180005', '330020', '550001']
{'510005': {}, '400009': {}, '180005': {}, '330020': {}, '550001': {}}


,ID_Estacion,momento,RR6_Valor,HR_Valor,Ts_Valor,ff_Valor,RadGInst (watt/m2)
0,510005,2023-01-01 00:00:00,0.0,57.8,9.7,23.0,78.1
6,510005,2023-01-01 06:00:00,0.0,63.8,7.9,17.0,0.0
12,510005,2023-01-01 12:00:00,0.2,60.0,10.0,13.0,290.5
18,510005,2023-01-01 18:00:00,0.0,42.0,15.0,31.0,1085.2
24,510005,2023-01-02 00:00:00,0.0,52.9,12.4,22.0,70.8
...,...,...,...,...,...,...,...
43216,550001,2023-12-30 18:00:00,0.0,59.5,11.4,12.0,222.1
43222,550001,2023-12-31 00:00:00,0.0,62.7,10.8,15.0,128.5
43228,550001,2023-12-31 06:00:00,0.0,78.5,8.2,4.0,0.0
43234,550001,2023-12-31 12:00:00,0.0,40.0,12.0,9.0,431.0


In [34]:
d = bigDf.iloc[:,3:]

m = np.full((5,1), [0])
n = np.full((5,1), [0])

m = [-1.5, -1, 10, 0.5]
n = [-0.5, 1.5, 0, 3]

ext_d = pd.DataFrame({"Estimacion de energia solar": d.dot(n)})
ext_d2 = pd.DataFrame({"Estimacion de energia eolica": d.dot(m)})

display([bigDf.iloc[:,2:].max(), bigDf.iloc[:,2:].min()])

#Vector de pesos = Cantidad de columnas.
#Si quiero indicar cuales valores tienen más prioridad, da valores mas altos
#Ej si quiero Solar: valores mas altos en aspectos de radiacion. Considerar multiplicadores negativos para columnas que impacten de forma negativa
    #Ej radiacion(positivo), precipitacion(negativo), humedad(negativo)
    #Si quiero Eolico: viento(positivo, depende de condiciones que describi en el doc)... y asi.

result = pd.concat([bigDf, ext_d, ext_d2], axis=1)
display(result)

[RR6_Valor               33.4
 HR_Valor               100.0
 Ts_Valor                35.6
 ff_Valor                91.0
 RadGInst (watt/m2)    1361.6
 dtype: float64,
 RR6_Valor              0.0
 HR_Valor               5.0
 Ts_Valor             -16.2
 ff_Valor               0.0
 RadGInst (watt/m2)     0.0
 dtype: float64]

,ID_Estacion,momento,RR6_Valor,HR_Valor,Ts_Valor,ff_Valor,RadGInst (watt/m2),Estimacion de energia solar,Estimacion de energia eolica
0,510005,2023-01-01 00:00:00,0.0,57.8,9.7,23.0,78.1,219.95,172.65
1,510005,2023-01-01 01:00:00,NaN,61.6,8.8,22.0,7.7,5.50,122.65
2,510005,2023-01-01 02:00:00,NaN,62.0,8.1,20.0,0.0,-18.85,98.90
3,510005,2023-01-01 03:00:00,NaN,59.6,8.7,20.0,0.0,-16.75,101.90
4,510005,2023-01-01 04:00:00,NaN,58.8,8.9,17.0,0.0,-16.05,72.90
...,...,...,...,...,...,...,...,...,...
43241,550001,2023-12-31 19:00:00,NaN,31.0,13.7,12.0,628.8,1891.45,374.20
43242,550001,2023-12-31 20:00:00,NaN,30.0,14.0,16.0,833.5,2506.50,517.75
43243,550001,2023-12-31 21:00:00,NaN,30.0,14.0,14.0,655.5,1972.50,408.75
43244,550001,2023-12-31 22:00:00,NaN,35.9,13.4,10.0,490.6,1473.95,278.05
